In [2]:
import sys
sys.path.append(r'D:\repositories\baltika_hack')

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Загрузка данных
failures = pd.read_excel('Простои за июль 2024.xlsx')
sensors = pd.read_excel('Датчики за июль 2024.xlsx')

# Преобразование временных меток в datetime
failures['failure_time'] = pd.to_datetime(failures['failure_time'], errors='coerce')
sensors['timestamp'] = pd.to_datetime(sensors['timestamp'], errors='coerce')

# Быстрый просмотр данных
print("\nFailures head:")
print(failures.head())
print(failures.info())

print("\nSensors head:")
print(sensors.head())
print(sensors.info())

# Проверка пропусков
print("\nMissing values in failures:")
print(failures.isnull().sum())

print("\nMissing values in sensors:")
print(sensors.isnull().sum())

# Распределение целевой переменной (если есть)
if 'failure_time' in failures.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(failures['failure_time'], kde=True)
    plt.title('Распределение времени до отказа')
    plt.show()

# Проверка корреляций (если есть числовые признаки)
numeric_features = sensors.select_dtypes(include=['float64', 'int64']).columns
plt.figure(figsize=(10, 8))
sns.heatmap(sensors[numeric_features].corr(), annot=True, cmap='coolwarm')
plt.title('Корреляция признаков датчиков')
plt.show()

# Объединение данных по equipment_id и ближайшим временным меткам
if 'equipment_id' in failures.columns and 'equipment_id' in sensors.columns:
    df = pd.merge_asof(
        sensors.sort_values('timestamp'),
        failures.sort_values('failure_time'),
        left_on='timestamp',
        right_on='failure_time',
        by='equipment_id',
        direction='backward'
    )
    print("\nMerged data head:")
    print(df.head())

# Сохранение промежуточного результата
df.to_csv('/mnt/data/merged_data.csv', index=False)

print("\nEDA завершён. Готов к следующему этапу!")


KeyError: 'failure_time'